In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from collections import defaultdict
import keras
from bn_model import TSN
from keras import backend as K

Using TensorFlow backend.


In [2]:
#Create folder for storing the frames
 
splitfiledir = r"..\ucfTrainTestlist"
splitfile = "trainlist01.txt"
splitname = splitfile.split('.')[0]

In [3]:
def storeFramesAndFlows(framesdir,splitfiledir,splitfile,splitname):
    
    #Read splitfile
    lines = open(os.path.join(splitfiledir,splitfile),"r")
    for line in lines:
        arr = line.split(" ")
        vidclass = arr[1] 
        line = arr[0].split("/")
        action = line[0]
        filename = line[1]
        if filename.split("_")[2] not in ('g08','g09'):
            continue
        actionpath = os.path.join(framesdir,splitname,action)
        framepath = os.path.join(actionpath,filename,"frames")
        flowpath = os.path.join(actionpath,filename,"flows")
        
        #Create folder for Action
        if not os.path.exists(actionpath):
            os.mkdir(actionpath)
        
        #Create folder for videofile
        if not os.path.exists(os.path.join(actionpath,filename)):
            os.mkdir(os.path.join(actionpath,filename))
        
        #Create folder for frames
        if not os.path.exists(framepath):
            os.mkdir(framepath)
            
        #Create folder for flows
        if not os.path.exists(flowpath):
            os.mkdir(flowpath)

        #Read video and collect frames, flows
        vidcap = cv2.VideoCapture(os.path.join(path,action,filename))
        count = 0 
        prevFrame = None
        nextFrame = None
        while True:
            success,image = vidcap.read()
            #Resize image to remain consistent with BN Inception model
            if not success:
                break
            image = cv2.resize(image,(224,224))
            frame = "frame_%d.jpg"%count
            flow_x = "flow_x_%d.jpg"%count
            flow_y = "flow_y_%d.jpg"%count
            framename = os.path.join(framepath,frame)
            flowname_x = os.path.join(flowpath,flow_x)
            flowname_y = os.path.join(flowpath,flow_y)
            if count == 0:
                prevFrame = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                cv2.imwrite(framename,image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                count += 1
                continue
            
            nextFrame = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            cv2.imwrite(framename,image)
            optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
            flow = optical_flow.calc(prevFrame, nextFrame, None)
            prevFrame = nextFrame
            flow[...,0] = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flow[...,1] = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
            cv2.imwrite(flowname_x,flow[...,0])
            cv2.imwrite(flowname_y,flow[...,1])

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            count += 1
            
        filename = os.path.join(actionpath,filename,"info.txt")
        #Store the frames count in txt file
        rate = open(filename,"w")
        rate.write("frames:"+str(count))
        rate.write("\n")
        rate.write("class:"+vidclass)
        rate.close()

        #Close the video object
        vidcap.release()

    print ("Stored")

In [5]:
#Store Frames
framesdir = r"../FramesFlows"

path = r"../../UCF-101"
# path = r"E:\capstone_adbi_data\UCF-101"

if not os.path.exists(os.path.join(framesdir,splitname)):
    os.mkdir(os.path.join(framesdir,splitname))
    storeFramesAndFlows(framesdir,splitfiledir,splitfile,splitname)
else:
    print ("Frames stored already!")

Frames stored already!


In [5]:
def getFF(classpath):
    frames = list()
    xflows = list()
    yflows = list()
    #Stack up frames and flows
    min_frames = float('inf')
    min_flows = float('inf')
    y_true = None
    for folder in os.listdir(classpath):
        folderpath = os.path.join(classpath,folder)
        # infopath = folderpath\info.txt
        infopath = os.path.join(folderpath,"info.txt")
        # Read info.txt for extracting class
        f = open(infopath,"r")
        y_true = int(f.readlines()[1].strip().split(':')[1])

        # Collect frames
        imgpath = os.path.join(folderpath,"frames")
        flowspath = os.path.join(folderpath,"flows")

        allframes = os.listdir(imgpath)
        allflows = os.listdir(flowspath)
        min_frames = min(len(allframes),min_frames)

        #Sort frames sequentially
        allframes.sort(key = lambda x: int(x.split('_')[1].split('.')[0]))

        #Sort flows sequentially
        allxflows = list(filter(lambda k: k.split('_')[1] == 'x',allflows))
        allyflows = list(filter(lambda k: k.split('_')[1] == 'y',allflows))
        allxflows.sort(key = lambda x: int(x.split('_')[2].split('.')[0]))
        allyflows.sort(key = lambda x: int(x.split('_')[2].split('.')[0]))
        min_flows = min(len(allxflows),min_flows)

        stack = list()
        for frame in allframes:
            img = cv2.imread(os.path.join(imgpath,frame))
            grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            grayimg = np.expand_dims(grayimg,axis = 2)
            stack.append(grayimg)
        frames.append(np.array(stack))
        
        
        #Collect flows
        xstack = list()
        ystack = list()
        for xflow,yflow in zip(allxflows,allyflows):
            xfl = cv2.imread(os.path.join(flowspath,xflow))
            yfl = cv2.imread(os.path.join(flowspath,yflow))
            grayxfl = cv2.cvtColor(xfl,cv2.COLOR_BGR2GRAY)
            grayyfl = cv2.cvtColor(yfl,cv2.COLOR_BGR2GRAY)
            grayxfl = np.expand_dims(grayxfl,axis = 2)
            grayyfl = np.expand_dims(grayyfl,axis = 2)
            xstack.append(grayxfl)
            ystack.append(grayyfl)
        xflows.append(np.array(xstack))
        yflows.append(np.array(ystack))

    return frames, xflows, yflows, y_true


In [6]:
dfpath = r"../FramesFlows/mytest"
frames_dict = dict()
xflows_dict = dict()
yflows_dict = dict()
for each in os.listdir(dfpath):
    classpath = os.path.join(dfpath,each)
    frames, xflows, yflows, class_name = getFF(classpath)
    frames_dict[class_name] = frames
    xflows_dict[class_name] = xflows
    yflows_dict[class_name] = yflows


In [7]:
#Dictionary key is the class
#Second index represents video number in that class (starting from 0)
#Shape shows NO_OF_FRAMES*HEIGHT_OF_IMAGE*WIDTH_OF_IMAGE
frames_dict[2][0].shape

(149, 224, 224, 1)

In [8]:
#Decided to keep number of segments as 3 for now (consistent with the paper)
segments = 3

In [4]:
def one_hot_encode(data, classes = 101):
    """
    :param data: data to be one hot encoded
    :return: np array with one hot encoding
    """
    labels = np.zeros((data.size, classes))
    labels[np.arange(data.size), data - 1] = 1
    return labels

def getSegments(frames_dict,segments=3):
    #Get k(equal to segments) random samples(snippets) from frames
    framesSegments = defaultdict(list)
    ySegments = defaultdict(list)
    np.random.seed(0)
    for class_name in frames_dict.keys():
        videos = frames_dict[class_name]
        for video in videos:
            vsegs = np.array_split(video,segments)
            for x in range(len(vsegs)):
                idx = np.random.randint(vsegs[x].shape[0])
                #Append random snippet to segments dictionary
                framesSegments[x].append(vsegs[x][idx])
                ySegments[x].append(class_name)
    
    #One hot encoding
    for x in range(segments):
        ySegments[x] = one_hot_encode(np.array(ySegments[x]))

    return framesSegments, ySegments

In [10]:
framesSegments, yFramesTrueSegments = getSegments(frames_dict)
xFlowsSegments, yFlowsTrueSegments = getSegments(xflows_dict)
yFlowsSegments, yFlowsTrueSegments = getSegments(yflows_dict)

In [11]:
#key is segment number(0,1...segments) and second index in the image
framesSegments[0][1].shape
#key is segment number(0,1,...segments) and second index returns one hot encoded class for the respective image
# yFramesTrueSegments[2][4]

(224, 224, 1)

In [12]:
def getMixedSegments(x,y):
    final = dict()
    for key in x.keys():
        mylist = list()
        for a,b in zip(x[key],y[key]):
            new = np.stack((a,b),axis = 2)
            new = np.squeeze(new,axis = 3)
            mylist.append(new)
        final[key] = mylist
    return final

In [13]:
flows = getMixedSegments(xFlowsSegments, yFlowsSegments)

In [84]:
class dataGenerator(keras.utils.Sequence):

    def __init__(self, filepath, batch_size, ffpath, segments = 3, test=False):
        self.filenames = list()
        self.labels = list()
        self.batch_size = batch_size
        self.filepath = filepath
        self.ffpath = ffpath
        self.segments = segments
        
        with open(self.filepath,"r") as f:
            for line in f.readlines():
                arr = line.split(" ")
                self.filenames.append(arr[0])
                self.labels.append(int(arr[1].strip()))
        
    def __len__(self):
        return len(self.filenames)//self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]
        batch_y = self.labels[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.filenames))]

        Xframes = defaultdict(list)
        Y = list()
        Xflows = defaultdict(list)
        for index,each in enumerate(batch_x):
            infopath = os.path.join(self.ffpath,each,"info.txt")
            imgpath = os.path.join(self.ffpath,each,"frames")
            flowspath = os.path.join(self.ffpath,each,"flows")
            f = open(infopath,"r")
            total_frames = int(f.readlines()[0].strip().split(':')[1])
            f.close()
            idxs = []
            base = total_frames//self.segments
            low = 1
            for _ in range(self.segments):
                high = min(low + base, total_frames)
                idxs.append(np.random.randint(low, high,1)[0])
                low = high + 1 
            frames = self.getFrames(idxs, imgpath)
            flows = self.getFlows(idxs, flowspath)
            for i in range(self.segments):
                Xframes[i].append(frames[i])
                
            for i in range(self.segments):
                Xflows[i].append(flows[i])
            
            Y.append(batch_y[index])
            
        finalX = dict()
        i = 1
        for key in Xframes.keys():
            finalX['input_'+str(i)] = np.array(Xframes[key])
            i += 1
        for key in Xflows.keys():
            finalX['input_'+str(i)] = np.array(Xflows[key])
            i += 1
            
        finalY = {'output':self.one_hot_encode(np.array(Y))}

        return (finalX,finalY)
    
    def one_hot_encode(self,data, classes = 101):
        """
        :param data: data to be one hot encoded
        :return: np array with one hot encoding
        """
        labels = np.zeros((data.size, classes))
        labels[np.arange(data.size), data - 1] = 1
        return labels


    def getFlows(self,idxs, flowspath):

        stack = list()
        for i in idxs:
            f1 = "flow_x_"+str(i)+".jpg"
            f2 = "flow_y_"+str(i)+".jpg"
            grayx = self.readImg(os.path.join(flowspath,f1))
            grayy = self.readImg(os.path.join(flowspath,f2))
            img = np.stack((grayx,grayy),axis = 2)
            img = np.squeeze(img,axis = 3)
            stack.append(img)
            
        return np.array(stack)
    
    def readImg(self,path):
        img = cv2.imread(path)
        grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        grayimg = np.expand_dims(grayimg,axis = 2)
        return grayimg
    
    
    def getFrames(self,idxs, imgpath):

        stack = list()
        for i in idxs:
            framename = "frame_"+str(i)+".jpg"
            stack.append(self.readImg(os.path.join(imgpath,framename)))
            
        return np.array(stack)

In [85]:
folderpath = "../ucfTrainTestlist"
filename = "trainlist01.txt"
dg = dataGenerator(os.path.join(folderpath,filename),4,os.path.join("../FramesFlows",filename.split(".")[0]))

In [80]:
# arr = dg.__getitem__(2)

In [33]:
#Total 6 inputs; First 3 are frames and last 3 flows
# arr[1]['average_3'].shape

In [40]:
#COMPILE MODEL
K.clear_session()
model = TSN()

In [41]:
model.compile(optimizer= keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit([np.array(framesSegments[0]),np.array(framesSegments[1]),np.array(framesSegments[2]),np.array(flows[0]),np.array(flows[1]),np.array(flows[2])],yFramesTrueSegments[0],batch_size = 3, epochs = 1)
np.random.seed(0)
#Fit generator
history = model.fit_generator(dg,epochs = 10)